# Mount GoogleDrive

We mount the GoogleDrive to have access to our json files with the annotations generated by using ImgLab.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Convert Annotations from COCO JSON format to Yolo TXT format

We use the following code (source: Chat GPT) to convert the annotations in COCO JSON format into the format suitable for YOLO.

We also tried the Python library "pylable", however, this library only offers conversion functionalities for bounding boxes used in object detection, but not for instance segmentation.

In [ ]:
import json

def coco_to_yolo(coco_file, output_file):
    with open(coco_file, 'r') as f:
        coco_data = json.load(f)

    annotations = coco_data['annotations']
    images = coco_data['images']
    categories = coco_data['categories']

    label_map = {category['id']: category['name'] for category in categories}

    with open(output_file, 'w') as f:
        for annotation in annotations:
            image_id = annotation['image_id']
            image_info = next((image for image in images if image['id'] == image_id), None)
            if image_info is None:
                continue

            width = image_info['width']
            height = image_info['height']

            file_name = image_info['file_name']
            image_path = file_name.split('/')[-1]  # Extracting just the image filename

            segmentation = annotation['segmentation'][0]  # Assuming single segmentation per annotation

            # Converting segmentation to YOLO format
            yolo_x = []
            yolo_y = []
            for i in range(0, len(segmentation), 2):
                yolo_x.append(segmentation[i] / width)
                yolo_y.append(segmentation[i + 1] / height)

            # Writing YOLO annotation to the output file
            f.write(f"{image_path} {label_map[annotation['category_id']]} {' '.join(map(str, yolo_x))} {' '.join(map(str, yolo_y))}\n")

    print("Conversion complete!")

# Usage example
coco_file = '/content/gdrive/MyDrive/Colab Notebooks/CV/Self-Annotated-Data/class2_coco_imglab.json'
output_file = '/content/gdrive/MyDrive/Colab Notebooks/CV/Self-Annotated-Data/class2_coco_imglab.txt'

coco_to_yolo(coco_file, output_file)


Conversion complete!


Afterwards, we manually change the txt file and add the label (classes: 0, 1, 2). The reason that we do this manually is that the categories that we input in ImgLab were not transfered to the json file. Also, we change the name of the txt files to fit the names of the images they belong to because these must be the same to work in YoloV8.